# Actividad: Implementación de Transformers para Clasificación de Textos

## Propósito de Aprendizaje
Adquirir habilidades prácticas en el uso de modelos avanzados de Transformers, específicamente BERT, para la clasificación de textos. Al finalizar, los estudiantes habrán desarrollado la capacidad de implementar, entrenar y evaluar un modelo BERT, y compararlo con modelos anteriores.

## Producto(s)
- **Cuaderno Jupyter (Jupyter Notebook):** Documentar el proceso completo de preprocesamiento, implementación, entrenamiento y evaluación del modelo BERT.
- **Informe (PDF/Markdown):** Un documento que resuma los hallazgos, incluyendo gráficos y análisis de las métricas de rendimiento.


## Ejercicio 1: Carga y Preprocesamiento de Datos

**Objetivo:** Familiarizarse con el conjunto de datos y prepararlos para el entrenamiento del modelo BERT.

**Acciones:**
1. Cargar el archivo `Noticias.xlsx` que contiene las noticias.
2. Preprocesar los datos, incluyendo tokenización y padding de las secuencias de texto.


In [ ]:
# Importar bibliotecas necesarias
# Importar bibliotecas necesarias
import pandas as pd
from transformers import BertTokenizer
import torch

# Cargar el conjunto de datos
file_path = '../../Datos/Datos Crudos/Noticias.xlsx'
data = pd.read_excel(file_path)

# Eliminar las filas con valores faltantes
data.dropna(inplace=True)

# Preprocesar los datos
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
data['input_ids'] = data['contenido'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
max_len = 128
data['input_ids'] = data['input_ids'].apply(lambda x: x[:max_len] + [0]*(max_len-len(x)) if len(x) < max_len else x[:max_len])

## Etiqueta para detectar deportes y filtrar noticias de archivo

data=data[data['Etiqueta']!="archivo"]

data['Etiqueta'] = [1 if x == "deportes" else 0 for x in data["Etiqueta"]]



# Dividir el conjunto de datos en entrenamiento y validación
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data['input_ids'].tolist(), data['Etiqueta'].tolist(), test_size=0.2, random_state=42)

# Convertir los datos a tensores
train_encodings = torch.tensor(X_train)
val_encodings = torch.tensor(X_val)
train_labels = torch.tensor(y_train.reshape(-1, 1))
val_labels = torch.tensor(y_val.reshape(-1, 1))

## attention masks

train_masks = torch.tensor([[float(i != 0) for i in ii] for ii in X_train])
val_masks = torch.tensor([[float(i != 0) for i in ii] for ii in X_val])


## Ejercicio 2: Implementación de BERT

**Objetivo**: Construir y entrenar un modelo BERT para clasificar las noticias.

**Acciones**:

1. Utilizar la biblioteca transformers de Hugging Face para cargar el modelo BERT preentrenado.
2. Ajustar el modelo con el conjunto de datos de entrenamiento.
3. Evaluar el modelo utilizando un conjunto de datos de validación.

In [ ]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "no_deportes", 1: "deportes"}
label2id = {"no_deportes": 0, "deportes": 1}

labels = list(id2label.keys())

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification",
                                                           num_labels=1)



In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 8
metric_name = "f1"



args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)


In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_dict({"input_ids": train_encodings, "attention_mask": train_masks, "labels": train_labels})
val_dataset = Dataset.from_dict({"input_ids": val_encodings, "attention_mask": val_masks, "labels": val_labels})

In [ ]:

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()



## Ejercicio 3: Análisis de Resultados
**Objetivo**: Evaluar y comparar el rendimiento del modelo BERT.

**Acciones**:

Calcular métricas de rendimiento como precisión, recall y F1-score.
Visualizar las curvas de aprendizaje y los resultados de validación.
Comparar los resultados con los modelos RNN y LSTM implementados anteriormente.

In [ ]:
outputs

In [ ]:
from sklearn.metrics import classification_report

# Evaluar el modelo
results = trainer.evaluate()
print(results)

# Calcular métricas adicionales
y_pred = trainer.predict(torch.utils.data.TensorDataset(val_encodings, val_labels)).predictions.argmax(axis=1)
print(classification_report(y_val, y_pred, target_names=data['Etiqueta'].unique()))

# Visualización de resultados
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(trainer.state.log_history)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Curvas de Aprendizaje - BERT')
plt.show()


## Ejercicio 4: Informe y Conclusiones

**Objetivo**: Documentar los resultados obtenidos y discutir las implicaciones prácticas.

**Acciones**:

Documentar los resultados obtenidos en un informe detallado, incluyendo gráficos y análisis de las métricas de rendimiento.
Discutir las implicaciones de los resultados para aplicaciones prácticas de NLP, como la automatización de la clasificación de noticias en sistemas de recomendación o análisis de medios.
Proponer posibles mejoras y futuras direcciones de investigación, como la exploración de arquitecturas híbridas o la integración de mecanismos de atención.